In [2]:
import openmc
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np


In [3]:
xMin = -5.; xMax = 5.;
yMin = -5.; yMax = 5.;
z_position = 0.01;

x_dist = openmc.stats.Uniform(xMin,xMax)
y_dist = openmc.stats.Uniform(yMin,yMax);
z_dist = openmc.stats.Discrete([z_position],[1.0]);

planar_dist = openmc.stats.CartesianIndependent(x_dist,y_dist,z_dist);

In [4]:
angle_dist = openmc.stats.Monodirectional((0.0,0.0,1.0));

In [10]:
settings = openmc.Settings()
energy = openmc.stats.Watt() # defaults to U-235 fission from thermal neutron
settings.source = openmc.Source(space=planar_dist,energy=energy, angle=angle_dist);

settings.run_mode = 'fixed source';
settings.batches = 100;
settings.particles = 30000;
settings.generations_per_batch = 1;


In [6]:
lead = openmc.Material(name='lead');
lead.add_element('Pb',1)
lead.set_density=('g/cm3',11.34);

air = openmc.Material(name='air');
air.add_element('O',0.2);
air.add_element('N',0.8);
air.set_density('g/cm3',1.3e-3);

materials = openmc.Materials([air,lead]);

In [8]:
length = 50.;# cm
width = 10.; # cm
height = 10.; # cm
box = openmc.model.RectangularParallelepiped(-width/2.,width/2.,
                                             -height/2.,height/2.0, 0., length,
                                             boundary_type='vacuum')                                            
fwall = openmc.ZPlane(z0=10.0);
bwall = openmc.ZPlane(z0=11.0);

detector = openmc.Sphere(x0=0., y0=0.,z0=15.0, r=2.0);


c1 = openmc.Cell()
c1.region = -box & -fwall
c1.fill = air

c2 = openmc.Cell()
c2.region = -box & +fwall & -bwall
c2.fill = lead

c3 = openmc.Cell()
c3.region = -box & +bwall & +detector
c3.fill = air

det = openmc.Cell()
det.region = -detector
det.fill = None

root_univ = openmc.Universe();
root_univ.add_cells([c1,c2,c3,det]);

geometry = openmc.Geometry();
geometry.root_universe = root_univ;

materials.export_to_xml()
geometry.export_to_xml()

In [9]:
flux = openmc.Tally(name='flux');
energy,dose = openmc.data.dose_coefficients('neutron','AP');
dose_filter = openmc.EnergyFunctionFilter(energy,dose);
surface_filter = openmc.SurfaceFilter(detector);

flux.filters = [dose_filter,surface_filter];
flux.scores = ['current'];
tallies = openmc.Tallies([flux]);
tallies.export_to_xml();